<a href="https://colab.research.google.com/github/pkrishna1801/AutoDefense/blob/main/Fine_tuning_RoBERTa_Unhealthy_Comment_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#The plan for fine-tuning RoBERTa with the UCC

### 1. Set up
- Install transformers and pytorch lightning and import required packages.
- Loads UCC data and inspect it.

### 2. Create a dataset

- Load data into pytorch dataset

### 3. Build the model
 - Create a new pytorch (lightning) model with the Hugging Face RoBERTa model and a multi-label classification head.

### 4. Test model performance
- Test model performance using the ROC AUC and same test-set used in the original paper.

In [1]:
!nvidia-smi

Sat Mar 15 01:38:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Imports

In [ ]:
%pip install torch --index-url https://download.pytorch.org/whl/cu121
%pip install transformers
%pip install pytorch-lightning
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install scikit-learn
%pip install tensorboard


Looking in indexes: https://download.pytorch.org/whl/cu121
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.5/780.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 7.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from transformers import (
    AutoModel,
    AutoTokenizer,
    get_cosine_schedule_with_warmup
)
from torch.utils.data import Dataset, DataLoader
from torch.optim import AdamW
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
from typing import Optional, Dict, List

# Check versions and CUDA availability
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"GPU count: {torch.cuda.device_count()}")

In [ ]:
train_path = 'data/ucc/train.csv'
val_path = 'data/ucc/val.csv'

In [ ]:
train_data = pd.read_csv(train_path)

In [ ]:
train_data.head(5)

# inspect data

In [ ]:
train_data['unhealthy'] = np.where(train_data['healthy'] == 1, 0, 1)

attributes = ['antagonize', 'condescending','dismissive','generalisation',
              'generalisation_unfair','hostile','sarcastic','unhealthy']

train_data[attributes].sum().plot.bar()

# Dataset

In [ ]:
class UCCDataset(Dataset):
    """Dataset class for Unhealthy Comment Corpus (UCC)

    Args:
        data_path (str): Path to the CSV data file
        tokenizer: HuggingFace tokenizer
        attributes (List[str]): List of target attributes to predict
        max_token_len (int): Maximum token length for padding/truncation
        sample (Optional[int]): Number of samples to use (for balanced sampling)
    """
    def __init__(
        self,
        data_path: str,
        tokenizer,
        attributes: List[str],
        max_token_len: int = 128,
        sample: Optional[int] = 5000
    ):
        self.tokenizer = tokenizer
        self.attributes = attributes
        self.max_token_len = max_token_len
        self.data = self._prepare_data(data_path, sample)

    def _prepare_data(self, data_path: str, sample: Optional[int]) -> pd.DataFrame:
        data = pd.read_csv(data_path)
        data['unhealthy'] = np.where(data['healthy'] == 1, 0, 1)

        # Convert all attribute columns to float type
        for attr in self.attributes:
            data[attr] = data[attr].astype(float)

        if sample:
            unhealthy = data[data[self.attributes].sum(axis=1) > 0]
            clean = data[data[self.attributes].sum(axis=1) == 0]
            return pd.concat([
                unhealthy,
                clean.sample(sample, random_state=42)
            ]).reset_index(drop=True)
        return data

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> Dict[str, torch.Tensor]:
        row = self.data.iloc[index]

        # Encode text
        encoding = self.tokenizer(
            str(row.comment),
            add_special_tokens=True,
            max_length=self.max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        # Get labels and ensure they are float type
        labels = torch.tensor(
            self.data.iloc[index][self.attributes].values.astype(float),
            dtype=torch.float
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': labels
        }

In [ ]:
model_name = 'roberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
ucc_ds = UCCDataset(train_path, tokenizer, attributes=attributes, max_token_len=128)
ucc_ds_val = UCCDataset(val_path, tokenizer, attributes=attributes, max_token_len=128, sample=None)

In [ ]:
ucc_ds.__getitem__(0)

In [ ]:
ucc_ds.__getitem__(0)['labels'].shape, ucc_ds.__getitem__(0)['input_ids'].shape, ucc_ds.__getitem__(0)['attention_mask'].shape

In [ ]:
len(ucc_ds)

In [ ]:
len(ucc_ds_val)

# Data module

In [ ]:
class UCCDataModule(pl.LightningDataModule):
    """PyTorch Lightning DataModule for UCC dataset

    Args:
        train_path (str): Path to training data
        val_path (str): Path to validation data
        attributes (List[str]): Target attributes to predict
        batch_size (int): Batch size for training
        max_token_length (int): Maximum token length
        model_name (str): HuggingFace model name
        num_workers (int): Number of workers for DataLoader
    """
    def __init__(
        self,
        train_path: str,
        val_path: str,
        attributes: List[str],
        batch_size: int = 32,
        max_token_length: int = 128,
        model_name: str = 'roberta-base',
        num_workers: int = 4
    ):
        super().__init__()
        self.train_path = train_path
        self.val_path = val_path
        self.attributes = attributes
        self.batch_size = batch_size
        self.max_token_length = max_token_length
        self.model_name = model_name
        self.num_workers = num_workers
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

    def setup(self, stage: Optional[str] = None):
        if stage in (None, "fit"):
            self.train_dataset = UCCDataset(
                self.train_path,
                self.tokenizer,
                self.attributes
            )
            self.val_dataset = UCCDataset(
                self.val_path,
                self.tokenizer,
                self.attributes,
                sample=None
            )

    def train_dataloader(self):
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True,
            pin_memory=True
        )

    def val_dataloader(self):
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False,
            pin_memory=True
        )

In [ ]:
ucc_data_module = UCCDataModule(
    train_path=train_path,
    val_path=val_path,
    attributes=attributes,
    batch_size=32,
    max_token_length=128,
    model_name='roberta-base',
    num_workers=4
)

In [ ]:
ucc_data_module.setup()

In [ ]:
ucc_data_module.train_dataloader()

In [ ]:
# Number of batches (9960 // 16 = 623)
len(ucc_data_module.train_dataloader())

# Model

In [ ]:
class UCCClassifier(pl.LightningModule):
    """Multi-label classifier for UCC using RoBERTa

    Args:
        config (Dict): Configuration dictionary containing model parameters
    """
    def __init__(self, config: Dict):
        super().__init__()
        self.save_hyperparameters(config)

        self.pretrained_model = AutoModel.from_pretrained(
            config['model_name'],
            return_dict=True
        )
        self.hidden = nn.Linear(
            self.pretrained_model.config.hidden_size,
            self.pretrained_model.config.hidden_size
        )
        self.classifier = nn.Linear(
            self.pretrained_model.config.hidden_size,
            config['n_labels']
        )
        self.dropout = nn.Dropout(p=0.1)
        self.loss_fn = nn.BCEWithLogitsLoss()

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.pretrained_model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled_output = torch.mean(outputs.last_hidden_state, 1)

        hidden_output = self.dropout(F.relu(self.hidden(pooled_output)))
        logits = self.classifier(hidden_output)

        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return loss, logits

    def training_step(self, batch):
        loss, _ = self(**batch)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch):
        loss, logits = self(**batch)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return {"val_loss": loss, "predictions": logits, "labels": batch["labels"]}

    def configure_optimizers(self):
        optimizer = AdamW(
            self.parameters(),
            lr=self.hparams.lr,
            weight_decay=self.hparams.weight_decay
        )

        scheduler = get_cosine_schedule_with_warmup(
            optimizer,
            num_warmup_steps=int(self.hparams.warmup * self.hparams.train_steps),
            num_training_steps=self.hparams.train_steps
        )

        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "step"
            }
        }

## Training setup

In [ ]:
# Set environment variable to avoid tokenizer warnings
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Configuration
config = {
    'model_name': 'distilroberta-base',
    'n_labels': len(attributes),
    'batch_size': 32,
    'lr': 2e-5,
    'warmup': 0.1,
    'weight_decay': 0.01,
    'n_epochs': 10,
    'train_steps': len(ucc_data_module.train_dataloader()) * 5  # total steps for 5 epochs
}

# Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints',
    filename='ucc-roberta-{epoch:02d}-{val_loss:.2f}',
    monitor='val_loss',
    mode='min',
    save_top_k=1
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    mode='min'
)

# Logger
logger = TensorBoardLogger("lightning_logs", name="ucc_classifier")

# Trainer
trainer = pl.Trainer(
    max_epochs=config['n_epochs'],
    accelerator='gpu',
    devices=1,  # Use single GPU
    callbacks=[checkpoint_callback, early_stopping],
    logger=logger,
    gradient_clip_val=1.0,
    log_every_n_steps=10
)

# Model
model = UCCClassifier(config)

# Initialize the model's pooler weights from pretrained
model.pretrained_model.init_weights()

# Train
trainer.fit(model, ucc_data_module)

# Predict with model

In [ ]:
def classify_raw_comments(model, dm):
    predictions = []
    model.eval()
    for batch in dm.val_dataloader():
        input_ids = batch['input_ids']
        attention_mask = batch['attention_mask']
        with torch.no_grad():
            _, logits = model(input_ids, attention_mask=attention_mask)  # Unpack loss and logits
            predictions.append(logits)
    flattened_predictions = torch.sigmoid(torch.cat(predictions, dim=0)).cpu().numpy()
    return flattened_predictions

In [ ]:
predictions = classify_raw_comments(model, ucc_data_module)

In [ ]:
val_data = pd.read_csv(val_path)
val_data['unhealthy'] = np.where(val_data['healthy'] == 1, 0, 1)
true_labels = np.array(val_data[attributes])

In [ ]:
from sklearn import metrics
plt.figure(figsize=(15, 8))
for i, attribute in enumerate(attributes):
  fpr, tpr, _ = metrics.roc_curve(
      true_labels[:,i].astype(int), predictions[:, i])
  auc = metrics.roc_auc_score(
      true_labels[:,i].astype(int), predictions[:, i])
  plt.plot(fpr, tpr, label='%s %g' % (attribute, auc))
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc='lower right')
plt.title('RoBERTa Trained on UCC Datatset - AUC ROC')